In [1]:
import json
import torch
import numpy as np

In [26]:
user_repo = {}
repo_inds = {}
repos = []
n = 100
with open('../contributors.json') as tmj:
    i = 0
    for tml in tmj.readlines()[:n]:
        line = json.loads(tml)
        if not line['repo'] in repo_inds:
            repo_inds[line['repo']] = i
            repos.append(line['repo'])
            i += 1
        else:
            print(line['repo'])
#         cntr = 0
        for user in line['contributors']:
            if not user['login'] in user_repo:
                user_repo[user['login']] = {}
            user_repo[user['login']][line['repo']] = user['contributions']
#             cntr += user['contributions']
#         print(len(line['contributors']),cntr)

In [34]:
for user in user_repo:
    total_contri = 0
    for repo in user_repo[user]:
        total_contri += user_repo[user][repo]
    for repo in user_repo[user]:
        user_repo[user][repo] /= total_contri

In [4]:
class SAE(torch.nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = torch.nn.Linear(len(repo_inds), 64)
        self.fc2 = torch.nn.Linear(64, len(repo_inds))
        self.activation = torch.nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x
cuda0 = torch.device('cuda:0')
sae = SAE()
sae.cuda(device=cuda0)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.RMSprop(sae.parameters(), lr = 0.001, weight_decay = 0.5)

In [5]:
nb_epoch = 10
batch_size = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for user in user_repo:
        input = torch.zeros(len(repo_inds),device=cuda0)
        for repo in user_repo[user]:
            input[repo_inds[repo]] = user_repo[user][repo]
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = len(repo_inds)/torch.sum(target.data > 0).item()
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            if s % batch_size = 0:
                optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 0.07228681675572259
epoch: 2 loss: 0.059526044882241606
epoch: 3 loss: 0.059334540945703694
epoch: 4 loss: 0.06234421338736745
epoch: 5 loss: 0.34641065070008437
epoch: 6 loss: 0.512658834244691
epoch: 7 loss: 1.3353166893898913
epoch: 8 loss: 1.6740623646956698
epoch: 9 loss: 3.0270266706877766
epoch: 10 loss: 1.8502485324082494


In [31]:
sae.eval()
input = torch.zeros(len(repo_inds),device=cuda0)
item = list(user_repo.items())[1]
print(item)
for repo in item[1]:
    input[repo_inds[repo]] = user_repo[item[0]][repo]
print(input)
print(sae(input))

('nwolek', {'jamoma/JamomaCore': 707})
tensor([707.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.], device='cuda:0')
tensor([ 0.9301,  0.8789,  0.9872,  1.8451,  0.8184,  6.3266, -5.6941,  5.8376,
         0.8607,  0.9516,  4.3175, -0.9121, -1.2570,  0.8614,  4.1121,  1.0031,
         0.8729,  0.9900,  0.9983,  0.8944,  0.8618,  0.9794,  0.8625,  0.8253,
         1.0058,  0.9667,  0.79

In [14]:
graph = {}
with open("../network.json") as nj:
    j = json.load(nj)
    nodes = j['nodes']
    aware = j['aware']
    for u1,g in enumerate(aware):
        n1 = nodes[u1]
        if not n1 in user_repo:
            continue
        if not n1 in graph:
            graph[n1] = {}
        for u in g:
            u2 = int(u)
            n2 = nodes[u2]
            if not n2 in user_repo:
                continue
            if not n2 in graph:
                graph[n2] = {}
            if not n2 in graph[n1]:
                graph[n1][n2] = 0
            if not n1 in graph[n2]:
                graph[n2][n1] = 0
            graph[n1][n2] += len(g[u])
            graph[n2][n1] += len(g[u])

In [16]:
for n1 in graph:
    ss = 0
    for n2 in graph[n1]:
        ss += graph[n1][n2]
    for n2 in graph[n1]:
        graph[n1][n2] /= ss

In [20]:
print(list(graph.items())[0])

('ilanbiala', {'tamird': 0.014285714285714285, 'othiym23': 0.014285714285714285, 'alanshaw': 0.014285714285714285, 'sebv': 0.014285714285714285, 'bcoe': 0.014285714285714285, 'jdalton': 0.014285714285714285, 'epelc': 0.014285714285714285, 'eddiemonge': 0.014285714285714285, 'ashleygwilliams': 0.014285714285714285, 'boneskull': 0.014285714285714285, 'zeke': 0.014285714285714285, 'ritch': 0.014285714285714285, 'snlacks': 0.014285714285714285, 'JulianWielga': 0.014285714285714285, 'Narretz': 0.014285714285714285, 'gkalpak': 0.02857142857142857, 'lukeis': 0.014285714285714285, 'dandv': 0.014285714285714285, 'admc': 0.014285714285714285, 'zhongsp': 0.014285714285714285, 'naomiblack': 0.014285714285714285, 'gdi2290': 0.02857142857142857, 'rgbkrk': 0.014285714285714285, 'nykolaslima': 0.014285714285714285, 'shyndman': 0.014285714285714285, 'SBoudrias': 0.02857142857142857, 'indexzero': 0.02857142857142857, 'marcysutton': 0.014285714285714285, 'Keats': 0.014285714285714285, 'EladBezalel': 0.01

In [39]:
u1 = 0
input = torch.zeros(len(repo_inds),device=cuda0)
item = list(user_repo.items())[u1]
print(item)
for repo in item[1]:
    input[repo_inds[repo]] = user_repo[item[0]][repo]
print(input)
pred = sae(input)
for n2 in graph[item[0]]:
    input = torch.zeros(len(repo_inds),device=cuda0)
    for repo in user_repo[n2]:
        input[repo_inds[repo]] = user_repo[n2][repo]
    output = sae(input)
    pred += output*graph[item[0]][n2]
print(pred)

('Nilson', {'jamoma/JamomaCore': 1.0})
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([  1.8558,   1.7540,   1.9687,   3.4959,   1.6331,  12.4999, -11.4527,
         11.5777,   1.7164,   1.8981,   8.6066,  -1.8578,  -2.6023,   1.7172,
          8.1961,   1.9991,   1.7396,   1.9732,   1.9891,   1.7827,   1.7176,
          1.9516,   1.7192,   1.6440,   2.0036,   1.9275,   1.5777,  -3.4062,
          1.7546,  -1.7206,  -8.3156,   1.9891,  18.8251,   1.8776,  -3.7470,
          1.8311,   1.9164,  -3.3923,   1.9682,   1.9937,   1.8113,   1.9477,
          1.8678,   1.6890,

In [22]:
def sort_to_k(ary,k,key=lambda x:x,reversed=False):
    k = min(k,len(ary))
    for i in range(k):
        for j in range(len(ary)-1-i):
            if not reversed:
                if key(ary[len(ary)-1-j]) < key(ary[len(ary)-2-j]):
                    ary[len(ary)-1-j],ary[len(ary)-2-j] = ary[len(ary)-2-j],ary[len(ary)-1-j]
            else:
                if key(ary[len(ary)-1-j]) > key(ary[len(ary)-2-j]):
                    ary[len(ary)-1-j],ary[len(ary)-2-j] = ary[len(ary)-2-j],ary[len(ary)-1-j]
    return ary

In [48]:
k = 30
pred = pred.cpu()
recs = sort_to_k(list(range(len(pred))),k,key=lambda i:pred[i],reversed=True)
for rec in recs[:k]:
    print(json.dumps((repos[rec],pred[rec].item())))

["npm/npm", 18.82508659362793]
["magnumripper/JohnTheRipper", 13.375268936157227]
["saltstack/salt", 13.04786205291748]
["aws/aws-sdk-java", 12.49986457824707]
["wallabag/wallabag", 11.577651977539062]
["irungentoo/toxcore", 11.292764663696289]
["NaturalNode/natural", 8.606616020202637]
["openMF/community-app", 8.196128845214844]
["rust-lang/cargo", 8.167122840881348]
["apache/flink", 8.051855087280273]
["philc/vimium", 7.411432266235352]
["boot2docker/boot2docker", 5.21930456161499]
["tomaka/glutin", 4.360992908477783]
["marklogic/node-client-api", 3.4958651065826416]
["mesosphere/marathon", 3.3996007442474365]
["sass/libsass", 2.186338186264038]
["owncloud/core", 2.011935234069824]
["angular/material", 2.004453420639038]
["yesodweb/yesod", 2.0035769939422607]
["lukasz-madon/awesome-remote-job", 2.0007212162017822]
["jekyll/jekyll", 1.9997336864471436]
["magento/magento2", 1.9991129636764526]
["KoffeinFlummi/AGM", 1.9970226287841797]
["aidancbrady/Mekanism", 1.9941158294677734]
["face